In [1]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import time
import xgboost as xgb
import os 
os.chdir("..")

In [4]:
import polars as pl

def compare_id_order(path_a: str, path_b: str, id_col: str = "Id", preview: int = 10) -> bool:
    # 先確認兩邊都有 Id 欄
    cols_a = set(pl.read_parquet(path_a, n_rows=0).columns)
    cols_b = set(pl.read_parquet(path_b, n_rows=0).columns)
    if id_col not in cols_a or id_col not in cols_b:
        raise ValueError(f"'{id_col}' 必須同時存在於兩個檔案；"
                         f"has_in_a={id_col in cols_a}, has_in_b={id_col in cols_b}")

    # 讀兩邊（Lazy），只取 Id，加上穩定 row index
    lf_a = (pl.scan_parquet(path_a)
              .select(pl.col(id_col).cast(pl.Int64).alias(id_col))
              .with_row_index("_i"))
    lf_b = (pl.scan_parquet(path_b)
              .select(pl.col(id_col).cast(pl.Int64).alias(f"{id_col}_b"))
              .with_row_index("_i"))

    # 先比對筆數
    n_a = lf_a.select(pl.len()).collect().item()
    n_b = lf_b.select(pl.len()).collect().item()
    if n_a != n_b:
        print(f"❌ 筆數不同：{path_a}={n_a:,} vs {path_b}={n_b:,}")
        return False

    # 逐列比對 Id（同索引）
    joined = lf_a.join(lf_b, on="_i", how="inner")
    diffs = joined.filter(pl.col(id_col) != pl.col(f"{id_col}_b")) \
                  .select(["_i", id_col, f"{id_col}_b"]) \
                  .collect()

    if diffs.height == 0:
        print("✅ 兩檔案 Id 完全相同且順序一致")
        return True
    else:
        print(f"❌ 發現 {diffs.height:,} 列 Id 不一致（顯示前 {preview} 列）：")
        print(diffs.head(preview))
        return False

# === 使用範例 ===
A = "data/data_from_json/test_final_reordered.parquet"
B = "data/test.parquet"
_ = compare_id_order(A, B)


✅ 兩檔案 Id 完全相同且順序一致


In [3]:
import polars as pl
import os

A_PATH   = "data/data_from_json/test_final.parquet"
B_PATH   = "data/test.parquet"
OUT_PATH = "data/data_from_json/test_final_reordered.parquet"

# 讀 A/B（Lazy），確保 Id 型別一致
lf_a = pl.scan_parquet(A_PATH).with_row_index("_a_i")
lf_a = lf_a.with_columns(pl.col("Id").cast(pl.Int64))

lf_b_ids = (
    pl.scan_parquet(B_PATH)
      .select(pl.col("Id").cast(pl.Int64).alias("Id"))
      .with_row_index("_ord")              # B 的目標順序
)

# 依 B 的順序重排 A；同一 Id 在 A 中若有多列，用 _a_i 維持原相對順序
lf_reordered = (
    lf_b_ids
    .join(lf_a, on="Id", how="left")
    .sort(["_ord", "_a_i"])
    .drop(["_ord", "_a_i"])
)

os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)
lf_reordered.sink_parquet(OUT_PATH, compression="zstd")
print("✅ saved:", OUT_PATH)

# （可選）檢查 B 有但 A 沒有 / A 有但 B 沒有 的 Id 數量
missing_in_a = lf_b_ids.join(lf_a.select("Id"), on="Id", how="anti").select(pl.len()).collect().item()
extra_in_a   = lf_a.join(lf_b_ids.select("Id"), on="Id", how="anti").select(pl.len()).collect().item()
print(f"🔎 Ids in B not in A: {missing_in_a}")
print(f"🔎 Ids in A not in B: {extra_in_a}")


✅ saved: data/data_from_json/test_final_reordered.parquet
🔎 Ids in B not in A: 0
🔎 Ids in A not in B: 0


In [7]:
import polars as pl
import os, re
from fnmatch import fnmatch

INPUT_PATH  = "data/data_from_json/test/test_merged.parquet"
OUTPUT_PATH = "data/data_from_json/test/test_merged_filtered.parquet"

# 你想「明確刪掉」的欄位（精確名稱）
EXTRA_DROP = [
    # 範例：填入你要刪的欄位名
    # "miniRules1_percentage",
    # "legs0_segments0_seatsAvailable",
    "position",
  "pointOfSale",
  "industryEN",
  "grade" 
]

# 你想用萬用字元匹配來刪的欄位（* ?）
DROP_WILDCARDS = [
    # 例如：全部 _overflow_json 結尾
    "*terminal",
    "*country*",
    "*_airport_city_code",
    "*airport_icao",
    # "legs1_segments1_*",
]

# 你想用 regex 刪的欄位（進階）
DROP_PATTERNS = [
    # r"^debug_",
    # r"_tmp$",
]

def build_keep_columns(all_cols: list[str]) -> list[str]:
    rx = [re.compile(p) for p in DROP_PATTERNS]
    keep = [     
        "legs0_segments2_departureFrom_airport_iata",
        "legs0_segments3_departureFrom_airport_iata",
        "legs1_segments2_departureFrom_airport_iata",
        "legs1_segments3_departureFrom_airport_iata"]
    for c in all_cols:
        # 1) 先排除 segments2 / segments3
        if ("segments2" in c) or ("segments3" in c):
            continue
        # 2) 精確名稱排除
        if c in EXTRA_DROP:
            continue
        # 3) 萬用字元排除
        if any(fnmatch(c, pat) for pat in DROP_WILDCARDS):
            continue
        # 4) regex 排除
        if any(r.search(c) for r in rx):
            continue
        keep.append(c)
    return keep

lf = pl.scan_parquet(INPUT_PATH)
keep_cols = build_keep_columns(lf.columns)
lf = lf.select(keep_cols)

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
lf.sink_parquet(OUTPUT_PATH, compression="zstd")
print("✅ saved:", OUTPUT_PATH, "cols kept:", len(keep_cols))


C:\Users\wani\AppData\Local\Temp\ipykernel_18032\2915300250.py:59: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  keep_cols = build_keep_columns(lf.columns)


✅ saved: data/data_from_json/test/test_merged_filtered.parquet cols kept: 168


In [2]:
# Load data
train_filled = pl.read_parquet('data/data_from_json/train/train_merged_filtered.parquet')
from scripts.for_json import enforce_ranker_constants

train_filled = enforce_ranker_constants(train_filled)
from scripts.feature_enigeer import clean_fill_and_cast_columns
train_filled = clean_fill_and_cast_columns(train_filled)

✅ 共找到 101 個字串欄位
✅ 共找到 67 個數值欄位
✅ 共找到 0 個布林欄位
🔍 尚未分類的欄位共有 1 個：
['requestDate']
✅ 完成：字串清理、數值 null→NaN、布林 0/1（保留布林 null）


In [3]:
train_filled = pl.read_parquet('data/data_from_json/train/train_merged_filtered_changed.parquet')
# train_filled = pl.read_parquet('data/train.parquet')


In [ ]:
from scripts.for_json import global_constant_features

# ===== 使用範例 =====
train_filled = pl.read_parquet("data/data_from_json/train/train_merged_filtered.parquet")
common_feats, coverage = global_constant_features(train_filled)
print("共同恆定欄位數量:", len(common_feats))
print(common_feats[:50])          # 先看前 50 個
print(coverage.head(10))         # 看每個欄位覆蓋幾個 ranker_id


In [ ]:
# Load data
test_filled = pl.read_parquet('data/data_from_json/test/test_merged_filtered.parquet')
from scripts.for_json import enforce_ranker_constants

test_filled = enforce_ranker_constants(test_filled)

from scripts.feature_enigeer import clean_fill_and_cast_columns
test_filled = clean_fill_and_cast_columns(test_filled)

✅ 共找到 99 個字串欄位
✅ 共找到 66 個數值欄位
✅ 共找到 0 個布林欄位
🔍 尚未分類的欄位共有 3 個：
['legs0_segments3_departureFrom_airport_iata', 'legs1_segments3_departureFrom_airport_iata', 'requestDate']
✅ 完成：字串清理、數值 null→NaN、布林 0/1（保留布林 null）


# Currency change

In [3]:
from scripts.feature_enigeer import convert_prices_to_rub_by_currency
# 需要統一成 RUB 的欄位
cols = [
    "totalPrice",
    "taxes",
    'pricingInfo_price',
    'pricingInfo_taxes',
    "miniRules0_monetaryAmount",
    "miniRules1_monetaryAmount",
]

# 例如：1 KZT = 0.20 RUB
train_filled = convert_prices_to_rub_by_currency(
    train_filled,
    cols_to_convert=cols,
    kzt_to_rub=0.20,
    in_place=True,      # 覆蓋原欄位
)

from scripts.feature_enigeer import apply_mini_rules

train_filled = apply_mini_rules(train_filled, percent_scale=100.0)
train_filled = train_filled.drop("currencyCode", "miniRules0_percentage", "miniRules1_percentage")
train_filled.write_parquet('data/data_from_json/train/train_merged_filtered_changed.parquet')


In [24]:
from scripts.feature_enigeer import convert_prices_to_rub_by_currency
# 需要統一成 RUB 的欄位
cols = [
    "totalPrice",
    "taxes",
    'pricingInfo_price',
    'pricingInfo_taxes',
    "miniRules0_monetaryAmount",
    "miniRules1_monetaryAmount",
]

# 例如：1 KZT = 0.20 RUB
test_filled = convert_prices_to_rub_by_currency(
    test_filled,
    cols_to_convert=cols,
    kzt_to_rub=0.20,
    in_place=True,      # 覆蓋原欄位
)

from scripts.feature_enigeer import apply_mini_rules

test_filled = apply_mini_rules(test_filled, percent_scale=100.0)
test_filled = test_filled.drop("currencyCode", "miniRules0_percentage", "miniRules1_percentage")
test_filled.write_parquet('data/data_from_json/test/test_merged_filtered_changed.parquet')


In [10]:
test_filled = pl.read_parquet('data/data_from_json/test/test_merged_filtered_changed.parquet')


# price rank

In [25]:
from scripts.feature_enigeer import build_price_features
# 執行價格特徵工程，並存檔
price_features = build_price_features(
    train_filled,
    output_dir="data/data_from_json/extra_features/train"
)

# 查看結果
print(price_features)


✅ 已完成價格特徵工程
✅ 已儲存 Parquet: data/data_from_json/extra_features/train\1_price_features.parquet
shape: (18_145_372, 8)
┌──────────┬────────────┬──────────┬───────────┬────────────┬────────────┬────────────┬────────────┐
│ Id       ┆ price_per_ ┆ tax_rate ┆ log_price ┆ totalPrice ┆ is_cheapes ┆ price_from ┆ price_perc │
│ ---      ┆ tax        ┆ ---      ┆ ---       ┆ _rank      ┆ t          ┆ _median_zs ┆ entile     │
│ i64      ┆ ---        ┆ f64      ┆ f64       ┆ ---        ┆ ---        ┆ core       ┆ ---        │
│          ┆ f64        ┆          ┆           ┆ u32        ┆ i8         ┆ ---        ┆ f64        │
│          ┆            ┆          ┆           ┆            ┆            ┆ f64        ┆            │
╞══════════╪════════════╪══════════╪═══════════╪════════════╪════════════╪════════════╪════════════╡
│ 17861657 ┆ 14.381456  ┆ 0.069488 ┆ 10.048107 ┆ 171        ┆ 0          ┆ 1.092533   ┆ 0.924766   │
│ 11791212 ┆ 18.955926  ┆ 0.052736 ┆ 10.969284 ┆ 39         ┆ 0          ┆ 1

In [3]:
from scripts.feature_enigeer import build_price_features
# 執行價格特徵工程，並存檔
price_features = build_price_features(
    test_filled,
    output_dir="data/data_from_json/extra_features/test"
)

# 查看結果
print(price_features)


✅ 已完成價格特徵工程
✅ 已儲存 Parquet: data/data_from_json/extra_features/test\1_price_features.parquet
shape: (6_897_776, 8)
┌──────────┬────────────┬──────────┬───────────┬────────────┬────────────┬────────────┬────────────┐
│ Id       ┆ price_per_ ┆ tax_rate ┆ log_price ┆ totalPrice ┆ is_cheapes ┆ price_from ┆ price_perc │
│ ---      ┆ tax        ┆ ---      ┆ ---       ┆ _rank      ┆ t          ┆ _median_zs ┆ entile     │
│ i64      ┆ ---        ┆ f64      ┆ f64       ┆ ---        ┆ ---        ┆ core       ┆ ---        │
│          ┆ f64        ┆          ┆           ┆ u32        ┆ i8         ┆ ---        ┆ f64        │
│          ┆            ┆          ┆           ┆            ┆            ┆ f64        ┆            │
╞══════════╪════════════╪══════════╪═══════════╪════════════╪════════════╪════════════╪════════════╡
│ 18984240 ┆ 94.049505  ┆ 0.010526 ┆ 9.159047  ┆ 2          ┆ 0          ┆ -0.672437  ┆ 0.037037   │
│ 18984246 ┆ 94.049505  ┆ 0.010526 ┆ 9.159047  ┆ 2          ┆ 0          ┆ -0.

# 處理duration

In [ ]:
import polars as pl
from scripts.feature_enigeer import build_duration_features
# 所需欄位
duration_cols = [
    "legs0_duration",
    "legs1_duration",
    "legs0_segments0_duration",
    "legs0_segments1_duration",
    "legs1_segments0_duration",
    "legs1_segments1_duration",
]

needed_cols = ["Id", "ranker_id", "totalPrice"] + duration_cols

# # 篩選
# train_filled = train_filled.select([c for c in needed_cols if c in train_filled.columns])

# 再交給 build_duration_features
duration_features = build_duration_features(
    train_filled.select(needed_cols),
    output_dir="data/data_from_json/extra_features/train/"
)


✅ 已輸出 Parquet: data/data_from_json/extra_features/train/2_duration_features.parquet


In [4]:
import polars as pl
from scripts.feature_enigeer import build_duration_features
duration_cols = [
    "legs0_duration",
    "legs1_duration",
    "legs0_segments0_duration",
    "legs0_segments1_duration",
    "legs1_segments0_duration",
    "legs1_segments1_duration",
]

needed_cols = ["Id", "ranker_id", "totalPrice"] + duration_cols

# 再交給 build_duration_features
duration_features = build_duration_features(
    test_filled.select(needed_cols),
    output_dir="data/data_from_json/extra_features/test/"
)


✅ 已輸出 Parquet: data/data_from_json/extra_features/test/2_duration_features.parquet


# frequent_flyer_features code

In [4]:
import polars as pl
from scripts.feature_enigeer import build_frequent_flyer_match_features
# 所需欄位
needed_cols = [
        "Id", "ranker_id", "frequentFlyer", "isVip",
        "legs0_duration", "legs1_duration",
        *[f"{s}_{t}" for s in [
            "legs0_segments0", "legs0_segments1", 
            "legs1_segments0", "legs1_segments1", 
        ] for t in ["marketingCarrier_code", "operatingCarrier_code", "duration"]]
    ]



# 再交給 build_duration_features
ff_features = build_frequent_flyer_match_features(
    train_filled.select([c for c in needed_cols if c in train_filled.columns]),
    output_dir="data/data_from_json/extra_features/train/"
)


✅ 已儲存 Parquet: data/data_from_json/extra_features/train/3_frequent_flyer_features.parquet
✅ 已完成 frequentFlyer / segment 自動偵測版本特徵生成


In [ ]:
import polars as pl
from scripts.feature_enigeer import build_frequent_flyer_match_features
# # 所需欄位
needed_cols = [
        "Id", "ranker_id", "frequentFlyer",
        "legs0_duration", "legs1_duration","isVip",
        *[f"{s}_{t}" for s in [
            "legs0_segments0", "legs0_segments1", "legs0_segments2", "legs0_segments3",
            "legs1_segments0", "legs1_segments1", "legs1_segments2", "legs1_segments3"
        ] for t in ["marketingCarrier_code", "operatingCarrier_code", "duration"]]
    ]


# 篩選

# 再交給 build_duration_features
ff_features = build_frequent_flyer_match_features(
    test_filled.select([c for c in needed_cols if c in test_filled.columns]),
    output_dir="data/data_from_json/extra_features/test/"
)


✅ 已儲存 Parquet: data/extra_features/test/3_frequent_flyer_features.parquet
✅ 已完成 frequentFlyer / segment 自動偵測版本特徵生成


# Bagges

In [13]:
from scripts.feature_enigeer import build_baggage_fee_features

baggage_fee_df = build_baggage_fee_features(
    df=train_filled.select([
        "Id",
        "totalPrice",
        'ranker_id',
        "legs0_segments0_baggageAllowance_quantity",
        "legs1_segments0_baggageAllowance_quantity",
        "miniRules0_monetaryAmount",
        "miniRules1_monetaryAmount",
        "miniRules0_statusInfos",
        "miniRules1_statusInfos"
    ]),
    output_dir="data/data_from_json/extra_features/train/"
)


✅ 已儲存 Parquet: data/data_from_json/extra_features/train/4_baggage_fee_features.parquet


In [ ]:
from scripts.feature_enigeer import build_baggage_fee_features

baggage_fee_df = build_baggage_fee_features(
    df=test_filled.select([
        "Id",
        "totalPrice",
        'ranker_id',
        "legs0_segments0_baggageAllowance_quantity",
        "legs1_segments0_baggageAllowance_quantity",
        "miniRules0_monetaryAmount",
        "miniRules1_monetaryAmount",
        "miniRules0_statusInfos",
        "miniRules1_statusInfos"
    ]),
    output_dir="data/data_from_json/extra_features/test/"
)


✅ 已儲存 Parquet: data/data_from_json/extra_features/test/4_baggage_fee_features.parquet


# cabinClass

In [ ]:
from scripts.feature_enigeer import build_cabin_features


cabin_features_df = build_cabin_features(
    df=train_filled.select([
        "Id","searchRoute",
        *[f"legs{i}_segments{j}_cabinClass" for i in [0,1] for j in range(2)],
        *[f"legs{i}_segments{j}_duration" for i in [0,1] for j in range(2)]
    ]),
    output_dir="data/data_from_json/extra_features/train/"
)


✅ 已儲存 Parquet: data/data_from_json/extra_features/train/5_cabin_features.parquet


In [27]:
from scripts.feature_enigeer import build_cabin_features


cabin_features_df = build_cabin_features(
    df=test_filled.select([
        "Id","searchRoute",
        *[f"legs{i}_segments{j}_cabinClass" for i in [0,1] for j in range(2)],
        *[f"legs{i}_segments{j}_duration" for i in [0,1] for j in range(2)]
    ]),
    output_dir="data/data_from_json/extra_features/test/"
)


✅ 已儲存 Parquet: data/data_from_json/extra_features/test/5_cabin_features.parquet


# add time

In [ ]:
from scripts.feature_enigeer import build_time_features

df_time_features = build_time_features(
    df=train_filled.select(["Id", "requestDate", "legs0_departureAt", "legs0_arrivalAt", "legs1_departureAt", "legs1_arrivalAt","searchRoute"]),
    output_dir="data/data_from_json/extra_features/train/"
)


✅ 已儲存 Parquet: data/data_from_json/extra_features/train/6_time_features.parquet


In [11]:
from scripts.feature_enigeer import build_time_features

df_time_features = build_time_features(
    df=test_filled.select(["Id", "requestDate", "legs0_departureAt", "legs0_arrivalAt", "legs1_departureAt", "legs1_arrivalAt","searchRoute"]),
    output_dir="data/data_from_json/extra_features/test/"
)


✅ 已儲存 Parquet: data/data_from_json/extra_features/test/6_time_features.parquet


# build_corporate_access_route_features

In [ ]:
from scripts.feature_enigeer import build_corporate_access_route_features
df_features = build_corporate_access_route_features(
    df=train_filled.select(["Id", "corporateTariffCode", "pricingInfo_isAccessTP", "searchRoute"]),
    output_dir="data/data_from_json/extra_features/train/"
)


✅ 已儲存 Parquet: data/data_from_json/extra_features/train/7_corporate_access_route_features.parquet
✅ 已完成 corporate/access/route 特徵生成（缺失值保留為 NaN）


In [12]:
from scripts.feature_enigeer import build_corporate_access_route_features
df_features = build_corporate_access_route_features(
    df=test_filled.select(["Id", "corporateTariffCode", "pricingInfo_isAccessTP", "searchRoute"]),
    output_dir="data/data_from_json/extra_features/test/"
)


✅ 已儲存 Parquet: data/data_from_json/extra_features/test/7_corporate_access_route_features.parquet
✅ 已完成 corporate/access/route 特徵生成（缺失值保留為 NaN）


# 加上是否直飛跟轉機幾次

In [10]:
from scripts.feature_enigeer import build_transfer_features
transfer_features_df = build_transfer_features(
    df=train_filled.select([
        "Id", "ranker_id", "legs0_duration", "legs1_duration",
        "legs0_segments1_departureFrom_airport_iata",
        "legs0_segments2_departureFrom_airport_iata",
        "legs0_segments3_departureFrom_airport_iata",
        "legs1_segments1_departureFrom_airport_iata",
        "legs1_segments2_departureFrom_airport_iata",
        "legs1_segments3_departureFrom_airport_iata"
    ]),
    output_dir="data/data_from_json/extra_features/train/"
)


✅ 已儲存 Parquet: data/data_from_json/extra_features/train/8_transfer_features.parquet
✅ 已完成轉機特徵生成


In [13]:
from scripts.feature_enigeer import build_transfer_features
transfer_features_df = build_transfer_features(
    df=test_filled.select([
        "Id", "ranker_id", "legs0_duration", "legs1_duration",
        "legs0_segments1_departureFrom_airport_iata",
        "legs0_segments2_departureFrom_airport_iata",
        "legs0_segments3_departureFrom_airport_iata",
        "legs1_segments1_departureFrom_airport_iata",
        "legs1_segments2_departureFrom_airport_iata",
        "legs1_segments3_departureFrom_airport_iata"
    ]),
    output_dir="data/data_from_json/extra_features/test/"
)


✅ 已儲存 Parquet: data/data_from_json/extra_features/test/8_transfer_features.parquet
✅ 已完成轉機特徵生成


# Carrier

In [ ]:
from scripts.feature_enigeer import build_carrier_consistency_features
required_columns = [
    # 主鍵
    "Id",
    "ranker_id","searchRoute",
    # legs0 轉機判斷
    "legs0_segments1_departureFrom_airport_iata",
    "legs0_segments2_departureFrom_airport_iata",
    "legs0_segments3_departureFrom_airport_iata",
    # legs1 轉機判斷
    "legs1_segments1_departureFrom_airport_iata",
    "legs1_segments2_departureFrom_airport_iata",
    "legs1_segments3_departureFrom_airport_iata",
    # legs0 Marketing
    "legs0_segments0_marketingCarrier_code",
    "legs0_segments1_marketingCarrier_code",
    # legs1 Marketing
    "legs1_segments0_marketingCarrier_code",
    "legs1_segments1_marketingCarrier_code",

]

import pickle
with open("data/data_from_json/extra_features/transform_config_rank.pkl", "rb") as f:
    config = pickle.load(f)

carrier_df = build_carrier_consistency_features(
    df=train_filled.select(required_columns),
    output_dir="data/data_from_json/extra_features/train/",
    transform_config=config
)


✅ 已儲存 Parquet: data/data_from_json/extra_features/train/9_carrier_consistency_features.parquet
✅ 主Carrier一致性/轉機次數特徵完成（單趟直接一致；encoding 未補 -1）


In [14]:
from scripts.feature_enigeer import build_carrier_consistency_features
required_columns = [
    # 主鍵
    "Id",
    "ranker_id","searchRoute",
    # legs0 轉機判斷
    "legs0_segments1_departureFrom_airport_iata",
    "legs0_segments2_departureFrom_airport_iata",
    "legs0_segments3_departureFrom_airport_iata",
    # legs1 轉機判斷
    "legs1_segments1_departureFrom_airport_iata",
    "legs1_segments2_departureFrom_airport_iata",
    "legs1_segments3_departureFrom_airport_iata",
    # legs0 Marketing
    "legs0_segments0_marketingCarrier_code",
    "legs0_segments1_marketingCarrier_code",

    # legs1 Marketing
    "legs1_segments0_marketingCarrier_code",
    "legs1_segments1_marketingCarrier_code",

]

test_filled = test_filled.select(required_columns)
import pickle
with open("data/data_from_json/extra_features/transform_config_rank.pkl", "rb") as f:
    config = pickle.load(f)

carrier_df = build_carrier_consistency_features(
    df=test_filled,
    output_dir="data/data_from_json/extra_features/test/",
    transform_config=config
)


✅ 已儲存 Parquet: data/data_from_json/extra_features/test/9_carrier_consistency_features.parquet
✅ 主Carrier一致性/轉機次數特徵完成（單趟直接一致；encoding 未補 -1）


# encoding category features.

In [5]:
from scripts.feature_enigeer import build_label_encoding_features
# ✅ 先把所有 columns 列出
all_cols = train_filled.columns

# ✅ Collect relevant columns
selected_cols = []
# ✅ Collect relevant columns + Id
selected_cols = ["Id"]  # 一定要先放Id

# Aircraft code
selected_cols += [c for c in all_cols if c.endswith("_aircraft_code")]

# Flight Number
selected_cols += [c for c in all_cols if c.endswith("_flightNumber")]

# Airport / City
selected_cols += [c for c in all_cols if "_arrivalTo_airport_" in c or "_departureFrom_airport_" in c]

# Carrier
selected_cols += [c for c in all_cols if c.endswith("_marketingCarrier_code") or c.endswith("_operatingCarrier_code")]

# searchRoute
selected_cols.append("searchRoute")

# frequentFlyer
selected_cols.append("frequentFlyer")
selected_cols.append("selected")


# 去除重複
selected_cols = list(dict.fromkeys(selected_cols))

print("✅ build_label_encoding_features 需要的欄位：", selected_cols)

# train_filled = train_filled.select(selected_cols)
# import pickle
# with open("data/extra_features/transform_config_rank.pkl", "rb") as f:
#     config = pickle.load(f)

# 執行編碼
df_encoded, config = build_label_encoding_features(
    df= train_filled.select(selected_cols),
    output_dir="data/data_from_json/extra_features/train/",
    # transform_config=config
)


✅ build_label_encoding_features 需要的欄位： ['Id', 'legs0_segments0_aircraft_code', 'legs0_segments1_aircraft_code', 'legs1_segments0_aircraft_code', 'legs1_segments1_aircraft_code', 'legs0_segments0_flightNumber', 'legs0_segments1_flightNumber', 'legs1_segments0_flightNumber', 'legs1_segments1_flightNumber', 'legs0_segments2_departureFrom_airport_iata', 'legs0_segments3_departureFrom_airport_iata', 'legs1_segments2_departureFrom_airport_iata', 'legs1_segments3_departureFrom_airport_iata', 'legs0_arrivalTo_airport_city_iata', 'legs0_arrivalTo_airport_code', 'legs0_arrivalTo_airport_iata', 'legs0_departureFrom_airport_city_iata', 'legs0_departureFrom_airport_code', 'legs0_departureFrom_airport_iata', 'legs0_segments0_arrivalTo_airport_city_iata', 'legs0_segments0_arrivalTo_airport_code', 'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments0_departureFrom_airport_city_iata', 'legs0_segments0_departureFrom_airport_code', 'legs0_segments0_departureFrom_airport_iata', 'legs0_segments1_arriv

In [29]:
from scripts.feature_enigeer import build_label_encoding_features
import pickle
all_cols = test_filled.columns

# ✅ Collect relevant columns
selected_cols = []
# ✅ Collect relevant columns + Id
selected_cols = ["Id"]  # 一定要先放Id

# Aircraft code
selected_cols += [c for c in all_cols if c.endswith("_aircraft_code")]

# Flight Number
selected_cols += [c for c in all_cols if c.endswith("_flightNumber")]

# Airport / City
selected_cols += [c for c in all_cols if "_arrivalTo_airport_" in c or "_departureFrom_airport_" in c]

# Carrier
selected_cols += [c for c in all_cols if c.endswith("_marketingCarrier_code") or c.endswith("_operatingCarrier_code")]

# searchRoute
selected_cols.append("searchRoute")

# frequentFlyer
selected_cols.append("frequentFlyer")

# 去除重複
selected_cols = list(dict.fromkeys(selected_cols))

print("✅ build_label_encoding_features 需要的欄位：", selected_cols)

with open("data/data_from_json/extra_features/transform_config_rank.pkl", "rb") as f:
    config = pickle.load(f)

# 執行編碼
df_encoded, config = build_label_encoding_features(
    df=test_filled.select(selected_cols),
    output_dir="data/data_from_json/extra_features/test/",
    transform_config = config
)


✅ build_label_encoding_features 需要的欄位： ['Id', 'legs0_segments0_aircraft_code', 'legs0_segments1_aircraft_code', 'legs1_segments0_aircraft_code', 'legs1_segments1_aircraft_code', 'legs0_segments0_flightNumber', 'legs0_segments1_flightNumber', 'legs1_segments0_flightNumber', 'legs1_segments1_flightNumber', 'legs0_segments2_departureFrom_airport_iata', 'legs0_segments3_departureFrom_airport_iata', 'legs1_segments2_departureFrom_airport_iata', 'legs1_segments3_departureFrom_airport_iata', 'legs0_segments0_arrivalTo_airport_city_iata', 'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments0_departureFrom_airport_iata', 'legs0_segments1_arrivalTo_airport_city_iata', 'legs0_segments1_arrivalTo_airport_iata', 'legs0_segments1_departureFrom_airport_iata', 'legs1_segments0_arrivalTo_airport_city_iata', 'legs1_segments0_arrivalTo_airport_iata', 'legs1_segments0_departureFrom_airport_iata', 'legs1_segments1_arrivalTo_airport_city_iata', 'legs1_segments1_arrivalTo_airport_iata', 'legs1_segments1

# add view time

In [ ]:
from scripts.feature_enigeer  import enrich_flight_view_features
# # 所需欄位 (補齊所有必要欄位)
needed_cols = [
    "Id",
    "ranker_id",
    # segment columns for legs0
    "legs0_segments0_departureFrom_airport_iata",
    "legs0_segments0_arrivalTo_airport_iata",
    "legs0_segments1_departureFrom_airport_iata",
    "legs0_segments1_arrivalTo_airport_iata",

    # segment columns for legs1
    "legs1_segments0_departureFrom_airport_iata",
    "legs1_segments0_arrivalTo_airport_iata",
    "legs1_segments1_departureFrom_airport_iata",
    "legs1_segments1_arrivalTo_airport_iata",

    "searchRoute"
]



# 呼叫 enrich_flight_view_features
flight_view_features, _ = enrich_flight_view_features(
    train_filled.select(needed_cols),
    output_dir="data/data_from_json/extra_features/train/",

)

C:\Users\wani\AppData\Local\Temp\ipykernel_6972\378240206.py:53: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df.melt(id_vars=[], value_vars=all_seg_key_cols)
C:\Users\wani\AppData\Local\Temp\ipykernel_6972\378240206.py:56: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias("segment_view_count"))
C:\Users\wani\AppData\Local\Temp\ipykernel_6972\378240206.py:96: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  leg0_counts = df.group_by("legs0_key").agg(pl.count().alias("leg0_flight_view_count"))
C:\Users\wani\AppData\Local\Temp\ipykernel_6972\378240206.py:97: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  leg1_counts = df.group_by("legs1_key").agg(pl.count().alias("leg1_fli

✅ 已儲存 flight view 特徵: data/data_from_json/extra_features/train/11_flight_view_features.parquet
✅ 已儲存 transform_config: data/data_from_json/extra_features/train/transform_flight_view_key_config.pkl


In [31]:
# from scripts.feature_enigeer  import enrich_flight_view_features
import pickle
# 所需欄位 (補齊所有必要欄位)
needed_cols = [
    "Id",
    "ranker_id",
    # segment columns for legs0
    "legs0_segments0_departureFrom_airport_iata",
    "legs0_segments0_arrivalTo_airport_iata",
    "legs0_segments1_departureFrom_airport_iata",
    "legs0_segments1_arrivalTo_airport_iata",

    # segment columns for legs1
    "legs1_segments0_departureFrom_airport_iata",
    "legs1_segments0_arrivalTo_airport_iata",
    "legs1_segments1_departureFrom_airport_iata",
    "legs1_segments1_arrivalTo_airport_iata",

    "searchRoute"
]



# 篩選

with open("data/data_from_json/extra_features/transform_flight_view_key_config.pkl", "rb") as f:
    config = pickle.load(f)
    
# 呼叫 enrich_flight_view_features
test_filled, _ = enrich_flight_view_features(
    test_filled.select(needed_cols),
    output_dir="data/data_from_json/extra_features/test/",
    transform_config=config
)

✅ 已儲存 flight view 特徵: data/data_from_json/extra_features/test/11_flight_view_features.parquet


# A make companyID

In [39]:
from scripts.feature_enigeer import make_companyID_into_features

_, transform_dict = make_companyID_into_features(
    df=train_filled,
    output_dir="data/data_from_json/extra_features/train/",
)

✅ 已儲存 transform_dict: data/data_from_json/extra_features/train/12_companyID_into_features.parquet
✅ 已儲存 transform_dict: data/data_from_json/extra_features/train/transform_dict_companyID.pkl


In [ ]:
from scripts.feature_enigeer import make_companyID_into_features
import pickle

with open("data/data_from_json/extra_features/transform_dict_companyID.pkl", "rb") as f:
    config = pickle.load(f)
    
# 呼叫 enrich_flight_view_features
test_filled, _ = make_companyID_into_features(
    test_filled,
    output_dir="data//data_from_json/extra_features/test/",
    transform_dict=config
)

✅ 已儲存 transform_dict: data/data_from_json/data_from_json/extra_features/test/12_companyID_into_features.parquet


# Transfer time

In [7]:
import polars as pl
import os
from typing import List, Optional

def build_transfer_duration_features(
    df: pl.DataFrame,
    *,
    transfer_cols: Optional[List[str]] = None,
    output_dir: Optional[str] = None,
    output_filename: str = "13_transfer_duration.parquet",
) -> pl.DataFrame:
    """
    - 直接把 transfer 欄位(字串 'HH:MM:SS' 或 'missing') 轉成分鐘(Float64)覆寫原欄位；缺值→NaN
    - 新增：
        transfer_duration_total  : 忽略 NaN 後的總分鐘；若全 NaN → 設 NaN
        transfer_duration_rank   : 依 ranker_id 內總分鐘升冪排名（越少越靠前=1）；total 為 NaN → rank 為 null
    """

    DEFAULT_TRANSFERS = [
        "legs0_segments0_transfer",
        "legs0_segments1_transfer",
        "legs1_segments0_transfer",
        "legs1_segments1_transfer",
    ]
    cols = transfer_cols or DEFAULT_TRANSFERS
    cols = [c for c in cols if c in df.columns]
    if not cols:
        # 沒有可處理的欄位就直接（可選）輸出 Id + 空新欄位
        out = df
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            out.write_parquet(os.path.join(output_dir, output_filename), compression="zstd")
        return out

    NAN = float("nan")

    def to_minutes_or_nan(col: str) -> pl.Expr:
        s = pl.col(col).cast(pl.Utf8)
        is_missing = s.is_null() | (s == "missing") | (s == "")
        # 解析 HH:MM:SS -> 分鐘（用 Float64 以便帶 NaN）
        hrs  = s.str.extract(r"^(\d+):", 1).cast(pl.Float64)
        mins = s.str.extract(r":(\d+):", 1).cast(pl.Float64)
        val = hrs * 60.0 + mins
        return pl.when(is_missing).then(pl.lit(NAN)).otherwise(val).alias(col)

    # 1) 覆寫原 transfer 欄位為分鐘（Float64；缺值→NaN）
    df2 = df.with_columns([to_minutes_or_nan(c) for c in cols])

    # 2) 總轉機時間（忽略 NaN；若全 NaN → NaN）
    valid_cnt = pl.sum_horizontal([pl.col(c).is_not_nan().cast(pl.Int8) for c in cols])
    sum_minutes = pl.sum_horizontal([pl.col(c).fill_nan(0.0) for c in cols])
    total_expr = (
        pl.when(valid_cnt > 0).then(sum_minutes).otherwise(pl.lit(NAN))
        .alias("transfer_duration_total")
    )
    df2 = df2.with_columns(total_expr)

    # 3) 排名（忽略 NaN → 將 NaN 先轉成 null 再 rank；時間越少 rank=1）
    total_for_rank = pl.when(pl.col("transfer_duration_total").is_nan()).then(None).otherwise(pl.col("transfer_duration_total")).alias("__total_for_rank")
    df2 = df2.with_columns(total_for_rank)

    if "ranker_id" in df2.columns:
        rank_expr = (
            pl.col("__total_for_rank")
              .rank(method="dense", descending=False)
              .over("ranker_id")
              .cast(pl.Int32)
              .alias("transfer_duration_rank")
        )
    else:
        rank_expr = (
            pl.col("__total_for_rank")
              .rank(method="dense", descending=False)
              .cast(pl.Int32)
              .alias("transfer_duration_rank")
        )

    df2 = df2.with_columns(rank_expr).drop("__total_for_rank","ranker_id")

    # 可選：輸出
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        df2.write_parquet(os.path.join(output_dir, output_filename), compression="zstd")
        print(f"✅ 已儲存：{os.path.join(output_dir, output_filename)}")

    return df2


In [12]:
transfer_col = ["legs0_segments0_transfer",
        "legs0_segments1_transfer",
        "legs1_segments0_transfer",
        "legs1_segments1_transfer",'ranker_id','Id']
features = build_transfer_duration_features(
    train_filled.select(transfer_col),
    output_dir="data/data_from_json/extra_features/train/"
)
features.head()


✅ 已儲存：data/data_from_json/extra_features/train/13_transfer_duration.parquet


legs0_segments0_transfer,legs0_segments1_transfer,legs1_segments0_transfer,legs1_segments1_transfer,Id,transfer_duration_total,transfer_duration_rank
f64,f64,f64,f64,i64,f64,i32
NaN,NaN,NaN,NaN,17861657,NaN,null
75.0,NaN,60.0,NaN,11791212,135.0,7
75.0,NaN,NaN,NaN,12886824,75.0,1
NaN,NaN,NaN,NaN,5937543,NaN,null
NaN,NaN,NaN,NaN,12283825,NaN,null


In [11]:
transfer_col = ["legs0_segments0_transfer",
        "legs0_segments1_transfer",
        "legs1_segments0_transfer",
        "legs1_segments1_transfer",'ranker_id','Id']
features = build_transfer_duration_features(
    test_filled.select(transfer_col),
    output_dir="data/data_from_json/extra_features/test/"
)
features.head()


✅ 已儲存：data/data_from_json/extra_features/test/13_transfer_duration.parquet


legs0_segments0_transfer,legs0_segments1_transfer,legs1_segments0_transfer,legs1_segments1_transfer,Id,transfer_duration_total,transfer_duration_rank
f64,f64,f64,f64,i64,f64,i32
NaN,NaN,NaN,NaN,18984240,NaN,null
NaN,NaN,NaN,NaN,18984246,NaN,null
NaN,NaN,NaN,NaN,18984251,NaN,null
NaN,NaN,NaN,NaN,18984276,NaN,null
625.0,NaN,NaN,NaN,18984311,625.0,3


# remove time

In [1]:
import os 
os.chdir("..")
from scripts.feature_enigeer import merge_original_with_extra_features, clean_fill_and_cast_columns,drop_constant_numeric_columns, replace_group_minmax_for_rankish_features

merged_df = merge_original_with_extra_features(
    base_parquet_path="data/data_from_json/train/train_merged_filtered_changed.parquet",
    extra_features_dir="data/data_from_json/extra_features/train"
)


# 如果要儲存
merged_df  = clean_fill_and_cast_columns(merged_df)
# merged_df = drop_constant_numeric_columns(merged_df,0.99)
# 直接覆寫 merged_df 中所有「欄名含 rank」的特徵為 group-wise Min–Max 值
# merged_df = replace_group_minmax_for_rankish_features(
#     merged_df, group_col="ranker_id", id_col="Id", constant_fill=0.5
# )
# merged_df.write_parquet("data/data_from_json/train_final.parquet")

✅ 讀取原始資料: data/data_from_json/train/train_merged_filtered_changed.parquet
✅ 共找到 13 個 Parquet 要合併
🔹 合併第 1/13 個: data/data_from_json/extra_features/train\1_price_features.parquet
🔹 合併第 2/13 個: data/data_from_json/extra_features/train\2_duration_features.parquet
⚠️ 6 個特徵將被新檔案覆蓋: ['legs1_segments1_duration', 'legs0_duration', 'legs1_segments0_duration', 'legs0_segments0_duration', 'legs1_duration', 'legs0_segments1_duration']
🔹 合併第 3/13 個: data/data_from_json/extra_features/train\3_frequent_flyer_features.parquet
⚠️ 25 個特徵將被新檔案覆蓋: ['ranker_id', 'legs0_segments1_operatingCarrier_code', 'total_duration_rank', 'legs1_segments1_marketingCarrier_code', 'legs1_duration_rank', 'isVip', 'legs1_segments1_duration', 'legs0_segments0_operatingCarrier_code', 'legs1_segments0_duration_rank', 'legs0_segments1_duration_rank', 'legs1_segments0_marketingCarrier_code', 'legs0_duration', 'legs0_segments1_duration', 'legs1_segments0_operatingCarrier_code', 'legs1_segments1_duration_rank', 'legs0_segments0_dur

In [1]:
import os 
os.chdir("..")

from scripts.feature_enigeer import merge_original_with_extra_features,clean_fill_and_cast_columns,replace_group_minmax_for_rankish_features
merged_df = merge_original_with_extra_features(
    base_parquet_path="data/data_from_json/test/test_merged_filtered_changed.parquet",
    extra_features_dir="data/data_from_json/extra_features/test"
    )

# 如果要儲存
merged_df  = clean_fill_and_cast_columns(merged_df)
# merged_df = merged_df.drop(['bySelf', 'legs0_segments2_baggageAllowance_quantity', 'legs0_segments2_baggageAllowance_weightMeasurementType', 'legs0_segments3_baggageAllowance_quantity', 'legs0_segments3_baggageAllowance_weightMeasurementType', 'legs0_segments3_seatsAvailable', 'legs1_segments2_baggageAllowance_quantity', 'legs1_segments2_baggageAllowance_weightMeasurementType', 'legs1_segments2_seatsAvailable', 'legs1_segments3_baggageAllowance_quantity', 'legs1_segments3_baggageAllowance_weightMeasurementType', 'legs1_segments3_seatsAvailable', 'miniRules0_percentage', 'miniRules1_percentage', 'pricingInfo_passengerCount', 'selected', 'legs0_segments2_marketingCarrier_code_in_ff', 'legs0_segments2_operatingCarrier_code_in_ff', 'legs0_segments3_marketingCarrier_code_in_ff', 'legs0_segments3_operatingCarrier_code_in_ff', 'legs1_segments2_marketingCarrier_code_in_ff', 'legs1_segments2_operatingCarrier_code_in_ff', 'legs1_segments3_marketingCarrier_code_in_ff', 'legs1_segments3_operatingCarrier_code_in_ff', 'legs0_segments3_duration_rank', 'legs1_segments2_duration_rank', 'legs1_segments3_duration_rank', 'legs0_segments3_cabinClass', 'legs1_segments2_cabinClass', 'legs1_segments3_cabinClass', 'legs0_segments3_duration', 'legs1_segments2_duration', 'legs1_segments3_duration', 'has_corporate_tariff', 'legs0_segments3_arrivalTo_airport_city_iata', 'legs0_segments3_arrivalTo_airport_iata', 'legs0_segments3_departureFrom_airport_iata', 'legs1_segments2_arrivalTo_airport_city_iata', 'legs1_segments2_arrivalTo_airport_iata', 'legs1_segments2_departureFrom_airport_iata', 'legs1_segments3_arrivalTo_airport_city_iata', 'legs1_segments3_arrivalTo_airport_iata', 'legs1_segments3_departureFrom_airport_iata', 'legs0_segments3_marketingCarrier_code', 'legs0_segments3_operatingCarrier_code', 'legs1_segments2_marketingCarrier_code', 'legs1_segments2_operatingCarrier_code', 'legs1_segments3_marketingCarrier_code', 'legs1_segments3_operatingCarrier_code', 'legs0_segments3_aircraft_code', 'legs1_segments2_aircraft_code', 'legs1_segments3_aircraft_code', 'legs0_segments3_flightNumber', 'legs1_segments2_flightNumber', 'legs1_segments3_flightNumber'])
# 直接覆寫 merged_df 中所有「欄名含 rank」的特徵為 group-wise Min–Max 值
# merged_df = replace_group_minmax_for_rankish_features(
#     merged_df, group_col="ranker_id", id_col="Id", constant_fill=0.5
# )

# merged_df.write_parquet("data/test_with_companyID_features.parquet")

✅ 讀取原始資料: data/data_from_json/test/test_merged_filtered_changed.parquet
✅ 共找到 13 個 Parquet 要合併
🔹 合併第 1/13 個: data/data_from_json/extra_features/test\1_price_features.parquet
🔹 合併第 2/13 個: data/data_from_json/extra_features/test\2_duration_features.parquet
⚠️ 6 個特徵將被新檔案覆蓋: ['legs0_duration', 'legs1_segments1_duration', 'legs1_duration', 'legs0_segments0_duration', 'legs0_segments1_duration', 'legs1_segments0_duration']
🔹 合併第 3/13 個: data/data_from_json/extra_features/test\3_frequent_flyer_features.parquet
⚠️ 25 個特徵將被新檔案覆蓋: ['legs0_duration', 'total_duration', 'isVip', 'frequentFlyer', 'ranker_id', 'legs1_segments0_operatingCarrier_code', 'legs0_segments1_marketingCarrier_code', 'legs1_segments0_duration', 'legs1_segments1_duration', 'legs0_segments0_duration_rank', 'legs0_duration_rank', 'legs0_segments0_duration', 'legs0_segments1_operatingCarrier_code', 'legs0_segments1_duration_rank', 'legs1_segments0_duration_rank', 'legs0_segments0_marketingCarrier_code', 'total_duration_rank', 'le

In [2]:

cols_to_drop = [
    "legs0_segments0_arrivalAt","legs0_segments0_class","legs0_segments0_departureAt",
    "legs0_segments0_marketingCarrier_alliance_code","legs0_segments0_ticketDesignator",
    "legs0_segments0_transfer","legs0_segments1_arrivalAt","legs0_segments1_class",
    "legs0_segments1_departureAt","legs0_segments1_marketingCarrier_alliance_code",
    "legs0_segments1_ticketDesignator","legs0_segments1_transfer","legs1_segments0_arrivalAt",
    "legs1_segments0_class","legs1_segments0_departureAt","legs1_segments0_marketingCarrier_alliance_code",
    "legs1_segments0_ticketDesignator","legs1_segments0_transfer","legs1_segments1_arrivalAt",
    "legs1_segments1_class","legs1_segments1_departureAt","legs1_segments1_marketingCarrier_alliance_code",
    "legs1_segments1_ticketDesignator","legs1_segments1_transfer","requestDepartureDate",
    "requestReturnDate","request_time","root_$id","supplierId","timeLimit","validatingCarrier",
    "frequentFlyer","legs0_departureAt","legs0_arrivalAt","legs1_departureAt","legs1_arrivalAt",
]
merged_df = merged_df.drop(cols_to_drop)
merged_df.write_parquet("data/data_from_json/train_final.parquet")


In [3]:
# 找字串欄
import polars as pl
str_cols = [c for c in merged_df.columns if merged_df[c].dtype in (pl.Utf8, pl.String)]
# 找數值欄
numeric_cols = [c for c in merged_df.columns if merged_df[c].dtype.is_numeric()]
# 找布林欄
bool_cols = [c for c in merged_df.columns if merged_df[c].dtype == pl.Boolean]

print(f"✅ 共找到 {len(str_cols)} 個字串欄位")
print(f"✅ 共找到 {len(numeric_cols)} 個數值欄位")
print(f"✅ 共找到 {len(bool_cols)} 個布林欄位")
known_cols = set(str_cols + numeric_cols + bool_cols)
other_cols = [c for c in merged_df.columns if c not in known_cols]

print(f"🔍 尚未分類的欄位共有 {len(other_cols)} 個：")
print(other_cols)



✅ 共找到 1 個字串欄位
✅ 共找到 279 個數值欄位
✅ 共找到 0 個布林欄位
🔍 尚未分類的欄位共有 1 個：
['requestDate']


In [3]:
import polars as pl

# 讀 parquet
test = pl.read_parquet('data/Try only companyID/test_filled.parquet')
train = pl.read_parquet('data/Try only companyID/train_filled.parquet')

# 比較欄位
train_cols = set(train.columns)
test_cols = set(test.columns)

print("✅ Train columns:", len(train_cols))
print("✅ Test columns:", len(test_cols))

print("\n🎯 Train 中有但 Test 沒有的欄位:")
print(sorted(train_cols - test_cols))

print("\n🎯 Test 中有但 Train 沒有的欄位:")
print(sorted(test_cols - train_cols))

if train_cols == test_cols:
    print("\n✅ Train 和 Test 欄位完全一致！")

# 比較 companyID
train_company_ids = set(train["companyID"].unique().to_list())
test_company_ids = set(test["companyID"].unique().to_list())

only_in_test = sorted(test_company_ids - train_company_ids)

print("\n🎯 Test 中有但 Train 沒有的 companyID:")
print(only_in_test)

print(f"\n✅ Test unique companyID: {len(test_company_ids)}")
print(f"✅ Train unique companyID: {len(train_company_ids)}")
print(f"✅ Test 但不在 Train 的 companyID 數量: {len(only_in_test)}")


✅ Train columns: 288
✅ Test columns: 290

🎯 Train 中有但 Test 沒有的欄位:
['selected']

🎯 Test 中有但 Train 沒有的欄位:
['companyID_mode_cabin_class', 'companyID_mode_has_transfer', 'companyID_mode_transfer_num']

🎯 Test 中有但 Train 沒有的 companyID:
[16880, 37962, 43019, 45683, 46743, 52514, 54908, 57310, 59070, 59090, 59277, 59487, 59818, 60219, 60288, 60602, 60838, 61579, 61607, 61627, 61641, 61654, 61709, 61724, 61751, 61810, 61818, 61822, 61832, 61874, 62020, 62029, 62098, 62110, 62368, 62459, 62463, 62794, 62871, 63196, 63243]

✅ Test unique companyID: 495
✅ Train unique companyID: 641
✅ Test 但不在 Train 的 companyID 數量: 41
